In [2]:
### Benjamin Tollison Hw6 ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate
import numpy as np
import matplotlib.pyplot as plt
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [3]:
x,h,rho,g,A,theta = symbols('x h rho g A theta')
c = cos(theta)
s = sin(theta)
T = Matrix([[c,s,0,0],[-s,c,0,0],[0,0,c,s],[0,0,-s,c]])
F_global = Matrix([0,(1/2)*rho*g*A*h,0,(1/2)*rho*g*A*h])
F_element = T @ F_global
displayEquations('F_element',F_element)

<IPython.core.display.Math object>

In [27]:
#### Problem 4 ####
    # element 1
rho = 0.3
def K_local(index):
    if index == [1,2]:
        E = 10**7
        delta_x = 100
        delta_y = 100
    elif index == [3,2]:
        E = 3*10**7
        delta_x = -100
        delta_y = 100
    A = 2
    L = np.sqrt(delta_x**2 + delta_y**2)
    c = delta_x/L
    s = delta_y/L
    K = E*A/L * np.array([[c*c,c*s,-c*c,-c*s],[c*s,s*s,-c*s,-s*s],[-c*c,-c*s,c*c,c*s],[-c*s,-s*s,c*s,s*s]])
    return K
def f_gravity_local(index):
    if index == [1,2]:
        delta_x = 100
        delta_y = 100
    elif index == [3,2]:
        delta_x = -100
        delta_y = 100
    A = 2
    rho = 0.3
    L = np.sqrt(delta_x**2 + delta_y**2)
    g = 9.81
    c = delta_x/L
    s = delta_y/L
    f = -0.5*A*g*L*rho * np.array([s,c,s,c])
    return f
# K_1 = K_local([1,2])
# K_2 = K_local([3,2])
# f_gravity1 = f_gravity_local([1,2])
# f_gravity2 = f_gravity_local([3,2])
gravity_global = np.array([0,-0.5*np.sqrt(2*100**2),0,-0.5*np.sqrt(2*100**2),0,-0.5*np.sqrt(2*100**2)])
F_loads = np.array([0,0,500,0,0,0])
def global_K(connectivity):
    K = np.zeros((6,6))
    for i in connectivity:
        K[i[0]*2-2][i[0]*2-2] += K_local(i)[0][0]
        K[i[0]*2-2][i[0]*2-2+1] += K_local(i)[0][1]
        K[i[0]*2-2][i[1]*2-2] += K_local(i)[0][2]
        K[i[0]*2-2][i[1]*2-2+1] += K_local(i)[0][3]
        
        K[i[0]*2-2+1][i[0]*2-2] += K_local(i)[1][0]
        K[i[0]*2-2+1][i[0]*2-2+1] += K_local(i)[1][1]
        K[i[0]*2-2+1][i[1]*2-2] += K_local(i)[1][2]
        K[i[0]*2-2+1][i[1]*2-2+1] += K_local(i)[1][3]
        
        K[i[1]*2-2][i[0]*2-2] += K_local(i)[2][0]
        K[i[1]*2-2][i[0]*2-2+1] += K_local(i)[2][1]
        K[i[1]*2-2][i[1]*2-2] += K_local(i)[2][2]
        K[i[1]*2-2][i[1]*2-2+1] += K_local(i)[2][3]
        
        K[i[1]*2-2+1][i[0]*2-2] += K_local(i)[3][0]
        K[i[1]*2-2+1][i[0]*2-2+1] += K_local(i)[3][1]
        K[i[1]*2-2+1][i[1]*2-2] += K_local(i)[3][2]
        K[i[1]*2-2+1][i[1]*2-2+1] += K_local(i)[3][3]
    return K
node_connections = [[1,2],[3,2]]
K = global_K(node_connections)
F_eq = gravity_global+ F_loads
# imposing kinematic constraints
dof_zeros = [0,1,4,5]
for i in dof_zeros:
    K[i][:] = 0
    K[:][i] = 0
    K[i][i] = 1
    F_eq[i] = 0
dof_global = np.linalg.inv(K) @ F_eq
# part a
for i in dof_global:
    print("[{}]".format(round(i,6)))

[-0.0]
[0.0]
[0.00219]
[0.000845]
[0.0]
[0.0]


In [28]:
# part b
K_vc = global_K(node_connections)
q = dof_global
Feq = gravity_global + F_loads
F_bc = K_vc@q - Feq
for i in F_bc:
    print("[{}]".format(round(i,6)))

[-214.644661]
[-143.933983]
[0.0]
[0.0]
[-285.355339]
[356.066017]


In [44]:
# part c
A = 2
simga_xx1 = 0.5*(1/np.sqrt(2) * F_bc[0] + 1/np.sqrt(2) * F_bc[1])
simga_xy1 = 0.5*(-1/np.sqrt(2) * F_bc[0] + 1/np.sqrt(2) * F_bc[1])
displayEquations('simga_xx1', simga_xx1)
displayEquations('simga_xy1', simga_xy1)
sigma_xx2 = (1/(2*np.sqrt(2))) * (-F_bc[4] + F_bc[5])
sigma_xy2 = (1/(2*np.sqrt(2))) * (-F_bc[4] - F_bc[5])
displayEquations('sigma_xx2', sigma_xx2)
displayEquations('sigma_xy2', sigma_xy2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [50]:
# part d
du_1 = (1/np.sqrt(2))*(dof_global[2]+dof_global[3])
displayEquations('du',du_1)
du_2 = -1/np.sqrt(2)*(dof_global[2]-dof_global[3])
displayEquations('du',du_2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [36]:
#### Problem 6 ####
P,g,L,omega,x = symbols('P g L omega x')
f_x = ((P*L)/(g)) * x * omega**2
f_y = P
N = Matrix([[1-x/L],[x/L]])
F_axial = sp.integrate(f_x*N, (x,0,L))
displayEquations('F_axial',F_axial)
F_normal = sp.integrate(f_y*N, (x,0,L))
displayEquations('F_normal',F_normal)
L_discrete = np.sqrt(30**2 + 10**2)
displayEquations('L_discrete',L_discrete)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
#### in class exercise ####
delta_x = 12 - 0 
delta_y = 12 - 0
L = np.sqrt(delta_x**2 + delta_y**2)
E = 10.5*10**6
t = .25
w = 1
A = t*w
I = 1/12 * w * t**3
c = delta_x / L
s = delta_y / L
K_41 = -(c**2*E*A)/L - (12*s**2*E*I)/L**3
print(K_41)

-77356.58800395232
